In [1]:
!pip install datasets
!pip install pip install transformers
!pip install tiktoken
!pip install wandb
!pip install tqdm
!unzip nanoGPT.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 95.0 MB/s eta 0:00

In [8]:
from datasets import load_dataset, load_metric
import random
import pandas as pd
import tiktoken

In [3]:
!python data/shakespeare_char/prepare.py

length of dataset in characters: 1,115,394
all the unique characters: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size: 65
train has 1,003,854 tokens
val has 111,540 tokens


In [4]:
!python train.py config/train_shakespeare_char.py

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 200
log_interval = 10 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 64
block_size = 256 # context of up to 256 previous characters

# baby GPT model :)
n_layer = 6
n_head = 6
n_embd = 384
dropout = 0.2

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 5000
lr_decay_iters = 5000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of 

In [5]:
!python sample.py --out_dir=out-shakespeare-char

Overriding: out_dir = out-shakespeare-char
number of parameters: 10.65M
Loading meta from data/shakespeare_char/meta.pkl...


ANGELO:
And cowards it be but back'd by your hands;
And what hands are the purpose to be determineed?

ISABELLA:
Even your mother, and full of men! What like eye
Have been mildly made overthrow me now?

ANGELO:
I know not, and thus: but if the princess, why he's
not known to live your highness than
A man of your pity sound more rivers and merry so more for us
To be thrusted so; so you must not know I this fault?

ISABELLA:
I shall too fish a day of into the hoice of a
craver-breath-changed vill
---------------

Men part, and grave with the easts of the sea-sense;
But when that she was contented with one.

JULIET:
I'll stay you to the case of the joy.

LADY CAPULET:
I think it strength in me; that's still the rest.

JULIET:
Sir, I shall not show it.
It is not the heavier and beggar,--

Nurse:
To make unlook'd upon the shore.

Nurse:
Living, if I should be brief, 

In [70]:
!python data/pubmed/prepare.py

Found cached dataset scientific_papers (/content/drive/MyDrive/109b project/dataset_cache/scientific_papers/pubmed/1.1.1/306757013fb6f37089b6a75469e6638a553bd9f009484938d8f75a4c5e84206f)
writing /content/data/pubmed/train.bin: 100% 1024/1024 [00:08<00:00, 126.20it/s]
writing /content/data/pubmed/val.bin:   6% 60/1024 [00:00<00:01, 550.53it/s]
Traceback (most recent call last):
  File "/content/data/pubmed/prepare.py", line 52, in <module>
    batch = dset.shard(num_shards=total_batches, index=batch_idx, contiguous=True).with_format('numpy')
  File "/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py", line 4583, in shard
    return self.select(
  File "/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py", line 543, in wrapper
    out: Union["Dataset", "DatasetDict"] = func(self, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/datasets/fingerprint.py", line 511, in wrapper
    out = func(dataset, *args, **kwargs)
  File "/usr/local/lib/python

In [72]:
!python train.py config/finetune_pubmed.py

Overriding config with config/finetune_pubmed.py:
import time

out_dir = 'out-pubmed'
# eval stuff
eval_interval = 100
eval_iters = 20
log_interval = 10
wandb_log = False # feel free to turn on
wandb_project = 'pubmed'
wandb_run_name = 'ft-' + str(time.time())

dataset = 'pubmed'
init_from = 'gpt2-large' # this is the largest GPT-2 model

# only save checkpoints if the validation loss improves
always_save_checkpoint = False

# the number of examples per iter:
# 1 batch_size * 32 grad_accum * 1024 tokens = 32,768 tokens/iter
# shakespeare has 301,966 tokens, so 1 epoch ~= 9.2 iters
batch_size = 6
gradient_accumulation_steps = 5 * 8

# finetune at constant LR
learning_rate = 3e-5
decay_lr = False

max_iters = 6000




tokens per iteration will be: 245,760
Initializing from OpenAI GPT-2 weights: gpt2-large
loading weights from pretrained gpt: gpt2-large
forcing vocab_size=50257, block_size=1024, bias=True
overriding dropout rate to 0.0
number of parameters: 772.72M
num decayed parameter t

In [79]:
dataset['abstract']

[" background : the present study was carried out to assess the effects of community nutrition intervention based on advocacy approach on malnutrition status among school - aged children in shiraz , iran.materials and methods : this case - control nutritional intervention has been done between 2008 and 2009 on 2897 primary and secondary school boys and girls ( 7 - 13 years old ) based on advocacy approach in shiraz , iran . \n the project provided nutritious snacks in public schools over a 2-year period along with advocacy oriented actions in order to implement and promote nutritional intervention . for evaluation of effectiveness of the intervention growth monitoring indices of pre- and post - intervention were statistically compared.results:the frequency of subjects with body mass index lower than 5% decreased significantly after intervention among girls ( p = 0.02 ) . \n however , there were no significant changes among boys or total population . \n the mean of all anthropometric in

# One nice feature of this is that we can specify the length of the article or abstract we want to generate from the text.

# Should make a wrapper that makes it easier to input article, section and abstract. 

# A short article ends up look a lot like an abstract

In [82]:
!python sample.py --out_dir=out-pubmed --start="ARTICLE STARTS HERE:" --num_samples=2 --max_new_tokens=1000

Overriding: out_dir = out-pubmed
Overriding: start = ARTICLE STARTS HERE:
Overriding: num_samples = 2
Overriding: max_new_tokens = 1000
number of parameters: 772.72M
No meta.pkl found, assuming GPT-2 encodings...
ARTICLE STARTS HERE:dysphagia is a common and severe problem in patients with type 1 diabetes . in general ,
dysphagia increases the risk of developing severe hypoglycemic episodes , but there is no clear relationship between glucose and dysphagia .
a study published in 2003 showed that the occurrence of dysphagia was higher in individuals with type 1 diabetes than in those without type 1 diabetes .
dysphagia results from a combination of protein production defects , hypolactatemia , hypoglycemic attacks , and electrolyte abnormalities .
dysphagia can be prevented by eating food that is rich in fiber and by taking dietary supplements or by adjusting the amount of physical activity .
however , studies on the effect of dietary supplements on glucose levels have not been performe